In [9]:
from utils import Ball, Agent, Environment, GameState, Binner
import numpy as np

position_diff_binner = Binner(-25, 25, 25)
ball_velocity_binner= Binner(-4, 4, 16)
action_binner = Binner(-1, 1, 4)

In [10]:
def get_reward(curr_state, next_state):
    curr_state_reward = abs(curr_state.desired_position - curr_state.ball_position)
    next_state_reward = abs(next_state.desired_position - next_state.ball_position)

    return next_state_reward - curr_state_reward

num_position_diff_bins = len(list(position_diff_binner.get_states()))
num_ball_velocity_bins = len(list(ball_velocity_binner.get_states()))

Q_table = np.zeros((num_position_diff_bins, num_ball_velocity_bins, num_position_diff_bins, num_ball_velocity_bins))
Q_table.shape

(25, 16, 25, 16)

In [13]:
Q_table[1, 2].shape

(25, 16)

In [46]:
from tqdm import tqdm

ball = Ball(10, 0)
agent1 = Agent(5, ball, 1)
agent2 = Agent(5, ball, 1)

desired_trajectory = [0] * 25 + [-10] * 25 + [10] * 25 + [0] * 25

env = Environment(ball, desired_trajectory, agent1, agent2)

num_iterations = 1_000
alpha = 0.9
gamma = 0.75

for _ in tqdm(range(num_iterations)):
    (_, _), _, done = env.reset()
    curr_state = env.get_state()
    while not done:
        action_bin = np.random.choice(action_binner.get_states())
        action = action_binner.get_val_from_bin(action_bin)
        (_, _), _, done = env.step([action, action])
        next_state = env.get_state() # .get_q_learning_state(position_diff_binner, ball_velocity_binner)

        reward = get_reward(curr_state, next_state)
        curr_state_binned = curr_state.get_q_learning_state(position_diff_binner, ball_velocity_binner)
        next_state_binned = next_state.get_q_learning_state(position_diff_binner, ball_velocity_binner)

        TD = reward + gamma * np.max(Q_table[next_state_binned[0], next_state_binned[1]]) - Q_table[curr_state_binned[0], curr_state_binned[1], next_state_binned[0], next_state_binned[1]]

        Q_table[curr_state_binned[0], curr_state_binned[1], next_state_binned[0], next_state_binned[1]] += alpha * TD
        curr_state = next_state

 36%|███▌      | 361/1000 [00:01<00:02, 234.14it/s]


IndexError: index 26 is out of bounds for axis 0 with size 25

In [43]:
def get_optimal_next_state(curr_state):
    print(Q_table[curr_state[0], curr_state[1]].shape)
    return np.argmax(Q_table[curr_state[0], curr_state[1]])

get_optimal_next_state([0, 0])

(25, 16)


0

array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
    